# Extração de Características para Texto


Documentos são apresentados em formato textual. Tal formato, não pode ser compreendido por uma máquina e portanto, deve ser adaptado. À esta adaptação, de um formato textual para um formato numérico, atribuímos o nome extração de características (_feature extraction_).

Antes de começarmos, alguns conceitos devem ser compreendidos:

* **Vocabulário:** Conjunto de palavras (tokens) únicos presentes no dataset.

## Preparando o Dataset

Antes de aplicarmos técnicas de extração de características devemos ter um conjunto de dados pré-processados.

In [2]:
import pandas

dataset = pandas.read_csv("../datasets/imbd-reviews/IMDB-reviews-dataset.csv", nrows=15)

### Remoção de Acentuação

In [3]:
import unidecode

def utf8_to_ascii(text):
    return unidecode.unidecode(text)

### Remoção de Tags HTML

In [4]:
import re

def delete_html_nodes(text):
    regex = re.compile("<.+>")
    
    return re.sub(regex, "", text)

### Tokenização

In [5]:
import spacy

def tokenize(corpus, deacc=True, trim_html=True, header="review"):
    nlp = spacy.load("en_core_web_md")
    
    tokens = []
    for index, row in corpus.iterrows():
        document = row[header]
        # remove accents
        if deacc:
            document = utf8_to_ascii(document)
        
        # remove HTML tags and its content
        if trim_html:
            document = delete_html_nodes(document)
        
        spacy_doc = nlp(document)
        
        tokens.append([token for token in spacy_doc])
            
    return tokens

### Remoção de Stop-Words

In [6]:
def remove_stop_words(corpus):
    _tokens = []
    index = -1
    for document in corpus:
        _tokens.append([])
        index += 1
        
        for token in document:
            if not token.is_stop:
                _tokens[index].append(token)
            
    return _tokens

### Lematização

In [7]:
def lemmatize(corpus, remove_punct=True, remove_digits=True):
    lemmatized = []
    index = -1
    for document in corpus:
        lemmatized.append([])
        index += 1
        
        for token in document:
            # punctuation removal
            if remove_punct and token.is_punct:
                continue
                
            # digits removal
            if remove_digits and token.is_digit:
                continue

            lemmatized[index].append(token.lemma_)
            
        lemmatized[index] = " ".join(lemmatized[index])
        
        
    return lemmatized

In [8]:
preprocessed_corpus = lemmatize(
        remove_stop_words(tokenize(
                dataset,
    
                deacc=True, trim_html=True)),
    
        remove_punct=True, remove_digits=True)

preprocessed_corpus[0]

'reviewer mention watch oz episode hooked right exactly happen main appeal fact go show dare forget pretty picture paint mainstream audience forget charm forget romance OZ mess episode see strike nasty surreal ready watch develop taste Oz get accustomed high level graphic violence violence injustice crooked guard sell nickel inmate kill order away mannered middle class inmate turn prison bitch lack street skill prison experience watch Oz comfortable uncomfortable view thats touch dark'

## Bag of Words

_Bag of Words_ ou BoW é uma representação simples, que baseia-se no número de ocorrências de cada token. Cada token único é armazenado numa estrutura chamada **vocabulário**, com uma posição fixa. O índice desta posição é utilizado para codificar os documentos: Cada novo documento posssui seus tokens rearranjados, de forma que apareçam na mesma ordem do vocabulário. Em seguida, cada token é substituído pelo número de vezes que aparece no documento. Tokens que estão presentes no vocabulário mas não no documento recebem valor 0.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer()
bow_vectorizer.fit(preprocessed_corpus)

CountVectorizer()

Abaixo podemos verificar o vocabulário formado para o dataset em estudo. Ao lado de cada termo consta seu índice.

In [16]:
# checking vocab
bow_vectorizer.vocabulary_

{'reviewer': 319,
 'mention': 236,
 'watch': 415,
 'oz': 266,
 'episode': 120,
 'hooked': 185,
 'right': 321,
 'exactly': 122,
 'happen': 174,
 'main': 227,
 'appeal': 17,
 'fact': 128,
 'go': 160,
 'show': 350,
 'dare': 86,
 'forget': 146,
 'pretty': 293,
 'picture': 279,
 'paint': 270,
 'mainstream': 228,
 'audience': 19,
 'charm': 58,
 'romance': 325,
 'mess': 237,
 'see': 339,
 'strike': 370,
 'nasty': 249,
 'surreal': 379,
 'ready': 306,
 'develop': 96,
 'taste': 383,
 'get': 157,
 'accustomed': 0,
 'high': 183,
 'level': 216,
 'graphic': 163,
 'violence': 409,
 'injustice': 196,
 'crooked': 81,
 'guard': 167,
 'sell': 342,
 'nickel': 253,
 'inmate': 197,
 'kill': 205,
 'order': 262,
 'away': 22,
 'mannered': 231,
 'middle': 238,
 'class': 62,
 'turn': 403,
 'prison': 295,
 'bitch': 36,
 'lack': 208,
 'street': 369,
 'skill': 357,
 'experience': 124,
 'comfortable': 68,
 'uncomfortable': 405,
 'view': 407,
 'thats': 391,
 'touch': 398,
 'dark': 87,
 'wonderful': 426,
 'little': 22

In [17]:
# Checking document representation
print(preprocessed_corpus[0])

bow_vectorizer.transform([preprocessed_corpus[0]]).toarray()

reviewer mention watch oz episode hooked right exactly happen main appeal fact go show dare forget pretty picture paint mainstream audience forget charm forget romance OZ mess episode see strike nasty surreal ready watch develop taste Oz get accustomed high level graphic violence violence injustice crooked guard sell nickel inmate kill order away mannered middle class inmate turn prison bitch lack street skill prison experience watch Oz comfortable uncomfortable view thats touch dark


array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 

### Problemas com o BoW

A representação BoW, embora simples e fácil de usar, apresenta alguns inconvenientes que podem inviabilizá-lo:

- Documentos muito extensos possuirão muitos termos repetidos e consequentemente valores muito elevados para alguns termos, podendo ocasionar em problemas no treinamento do modelo;
- Corpora muito grandes podem ocasionar em uma representação (demasiadamente) esparsa;
- A ordem das palavras é totalmente desconsiderada, o que elimina muito de seu significado;

## Tf-Idf (_Term frequency_ - _Inverse document frequency_)

Tf-Idf é uma técnica muito similar ao BoW, que tem por objetivo eliminar o problema de pesos muito elevados. Em suma, o valor de tf-idf para um termo $t$ de um documento $d$ aumenta na mesma que a quantidade de ocorrências de $t$ em $d$ aumenta, mas diminui se $t$ aparecer em muitos documentos. Esta propriedade permite que termos que sejam realmente importantes no corpus possuam valores mais altos.  

Trata-se de uma estatística que é composta pelo produto de outras duas: **tf** (_term frequency_) e **idf** (_inverse document frequency_). Há diversas formas de calcular estes valores, portanto abordaremos uma detre estas.

### _Term frequency_

Refere-se à frequência de ocorrências de um termo em um documento.

$$tf(t, d) = f_{t,d},$$

onde $f_{t,d}$ é o número de vezes que o termo $t$ aparece no documento $d$.

### _Inverse document frequency_

Refere-se à frequência de documentos que contém um termo.

$$idf(t, D) = log \frac{|D|}{|\{d \in D : t \in d\}|},$$

onde $D$ é a coleção completa de documentos.

----

O valor final será dado por:

$$tf\text{-}idf(t, d, D) = tf(t, d) \times idf(t, D)$$

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(preprocessed_corpus)

TfidfVectorizer()

Abaixo podemos verificar o vocabulário gerado para o dataset em estudo. Ele será muito similar (quando não igual) ao gerado por `CountVectorizer`.

In [14]:
# checking vocab
tfidf_vectorizer.vocabulary_

{'reviewer': 319,
 'mention': 236,
 'watch': 415,
 'oz': 266,
 'episode': 120,
 'hooked': 185,
 'right': 321,
 'exactly': 122,
 'happen': 174,
 'main': 227,
 'appeal': 17,
 'fact': 128,
 'go': 160,
 'show': 350,
 'dare': 86,
 'forget': 146,
 'pretty': 293,
 'picture': 279,
 'paint': 270,
 'mainstream': 228,
 'audience': 19,
 'charm': 58,
 'romance': 325,
 'mess': 237,
 'see': 339,
 'strike': 370,
 'nasty': 249,
 'surreal': 379,
 'ready': 306,
 'develop': 96,
 'taste': 383,
 'get': 157,
 'accustomed': 0,
 'high': 183,
 'level': 216,
 'graphic': 163,
 'violence': 409,
 'injustice': 196,
 'crooked': 81,
 'guard': 167,
 'sell': 342,
 'nickel': 253,
 'inmate': 197,
 'kill': 205,
 'order': 262,
 'away': 22,
 'mannered': 231,
 'middle': 238,
 'class': 62,
 'turn': 403,
 'prison': 295,
 'bitch': 36,
 'lack': 208,
 'street': 369,
 'skill': 357,
 'experience': 124,
 'comfortable': 68,
 'uncomfortable': 405,
 'view': 407,
 'thats': 391,
 'touch': 398,
 'dark': 87,
 'wonderful': 426,
 'little': 22

In [18]:
# Checking document representation
print(preprocessed_corpus[0])

tfidf_vectorizer.transform([preprocessed_corpus[0]]).toarray()

reviewer mention watch oz episode hooked right exactly happen main appeal fact go show dare forget pretty picture paint mainstream audience forget charm forget romance OZ mess episode see strike nasty surreal ready watch develop taste Oz get accustomed high level graphic violence violence injustice crooked guard sell nickel inmate kill order away mannered middle class inmate turn prison bitch lack street skill prison experience watch Oz comfortable uncomfortable view thats touch dark


array([[0.10162097, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.10162097, 0.        , 0.10162097,
        0.        , 0.        , 0.10162097, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.10162097, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.10162097, 0.        ,
        0.        , 0.        , 0.10162097, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.10162097, 0.        ,
        0.        , 0.        , 0.        , 0.  

## Referências

- [Machine Learning — Text Processing](https://towardsdatascience.com/machine-learning-text-processing-1d5a2d638958)

- [NLP with Python: Text Feature Extraction](https://sanjayasubedi.com.np/nlp/nlp-feature-extraction/)